In [ ]:
# Select a subset of keypoints for training

In [1]:
import os
import numpy as np
import pandas as pd


In [ ]:
selected_keypoints = ['Small_Tip', 'Samll_DIP', 'Small_PIP','Ring_Tip', 'Ring_DIP','Ring_PIP',
                     'Middle_Tip', 'Middle_DIP', 'Middle_PIP', 'Index_Tip', 'Index_DIP', 'Index_PIP']
# correct 'Small_DIP' in the annotation tool

In [ ]:
base_dir = r'E:\Hand_tracking\Datasets\6cam_dataset_231124'
save_dir = r'E:\Hand_tracking\Datasets\6cam_dataset_231127'
trials = [t for t in os.listdir(base_dir) if t.startswith("2023")]
for t in trials:
    cameras = os.listdir(os.path.join(base_dir,t)) 
    for c in cameras:
        csv_path = os.path.join(base_dir,t,c,'annotations.csv')
        df1 = pd.read_csv(csv_path, on_bad_lines='skip', header = None) 
        df2 = pd.read_csv(csv_path, skiprows=4, header = None) 
        df2 = df2.drop(columns=[70],axis='columns')
        df = pd.concat([df1,df2],ignore_index=True)
        
        # Find bodyparts row
        isBodyparts = [x == 'bodyparts' for x in df[0]]
        df_bodyparts = df[isBodyparts]
        # Select bodyparts in selected keypoints
        isSelected = [df_bodyparts.iloc[0, index] in selected_keypoints for index in range(df_bodyparts.shape[1])]
        df3 = df.iloc[:,isSelected]
        new_df = pd.concat([df[0],df3], axis=1,ignore_index=True)
        
        # Change entities MonkeyHand -> MonkeyFingers
        isEntities = [x == 'entities' for x in new_df[0]]
        new_entities = np.repeat('MonkeyFingers',36)
        new_df.iloc[isEntities,1:] = new_entities
        
        # Correct "Small_DIP" typo
        new_bodyparts = np.repeat('Small_DIP',3).reshape([1,3])
        new_df.iloc[2, 4:7] = new_bodyparts

        # save new csv
        new_csv_path = os.path.join(save_dir,t,c,'annotations.csv')
        new_df.to_csv(new_csv_path, index=False, header=False)